In [1]:
%load_ext rich

In [2]:
import math
import promptimus as pm
from tracing import Tracer
from rich import print

In [3]:
provider = pm.llms.OpenAILike(
    model_name="gemma3:12b",
    base_url="http://lilan:11434/v1",
    api_key="DUMMY",
)

In [4]:
def power(a: float, b: float) -> float:
    """Calcuates the `a` in the pover of `b`"""
    return a**b


def factorial(a: int) -> int:
    """Calcuates the factorial (!) of `a`"""
    return math.factorial(a)


def multiply(a: float, b: float) -> float:
    """Multiplies `a` and `b`"""
    return a * b

In [5]:
tool_calling = pm.modules.ToolCallingAgent(
    [power, multiply, factorial], observation_role=pm.MessageRole.USER
).with_llm(provider)

In [6]:
tracer = Tracer()

tracer.decorate(tool_calling)

In [7]:
await tool_calling.forward("What is twice the factorial of 3?")

Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='12.0', tool_calls=None, tool_call_id=None)

In [8]:
while not tracer.queue.empty():
    print(tracer.queue.get_nowait())

Span(
    span_id='c5a73a5a-d50e-4be7-a877-a9b957178c64',
    parent_id=None,
    module_name='root',
    module_digest='a31de4f795d2eed6e3dae3ba1896d4bc',
    start=datetime.datetime(2025, 6, 18, 11, 29, 11, 785450, tzinfo=datetime.timezone.utc),
    stop=None,
    request='What is twice the factorial of 3?',
    response=None,
    status=<LogStatus.RUNNING: 'RUNNING'>,
    parameters={},
    error=None
)

Span(
    span_id='b9aadd1c-b54d-4169-925e-2b76d4ad585e',
    parent_id='c5a73a5a-d50e-4be7-a877-a9b957178c64',
    module_name='predictor',
    module_digest='5909e943adbe3a8d55d2847f2c0d8dc3',
    start=datetime.datetime(2025, 6, 18, 11, 29, 11, 785523, tzinfo=datetime.timezone.utc),
    stop=None,
    request="What is twice the factorial of 3?\ntool_desc: ## `power` tool.\nCalcuates the `a` in the pover of 
`b`\n\nParameters:\n- `a`: float\n- `b`: float\n## `multiply` tool.\nMultiplies `a` and `b`\n\nParameters:\n- `a`: 
float\n- `b`: float\n## `factorial` tool.\nCalcuates the factorial (!) of `a`\n\nParameters:\n- `a`: 
int\ntool_names: ['power', 'multiply', 'factorial']",
    response=None,
    status=<LogStatus.RUNNING: 'RUNNING'>,
    parameters={},
    error=None
)

Trace(
    trace_id='d3975177-c1c6-4cd9-8d6b-87616dc6039a',
    parent_id='b9aadd1c-b54d-4169-925e-2b76d4ad585e',
    prompt_name='prompt',
    prompt_digest='a8edbbe0e29b507be1247289d410e3a2',
    start=datetime.datetime(2025, 6, 18, 11, 29, 11, 785574, tzinfo=datetime.timezone.utc),
    stop=None,
    prompt='\nYou are designed to assist with a wide range of tasks—from answering questions and providing 
summaries to performing detailed analyses—by utilizing a variety of external tools. Follow these strict 
instructions to ensure correct tool usage and response formatting:\n\n---\n\n## Tools\n\n- **Tool Access:**  \n  
You have access to multiple tools: \n\n  {tool_desc}\n\n- **Execution Protocol:**  \n  - **One Step at a Time:** In
each response, you must either make a single tool call or provide a direct answer to the user.\n  - **No 
Fabrication:** You are strictly forbidden from generating any `Observation:` lines or simulating tool outputs. Do 
not assume, infer, or create tool responses—always wait for the actual output from the external tool.\n\n---\n\n## 
Output Format\n\nWhen processing a user\'s query, follow this structured format:\n\n1. **Initial Response (Tool 
Call or Direct Answer):**  \n   - **Thought:** Start every response with a clear thought outlining your reasoning. 
\n   - **Tool Call:**  \n     If you decide that a tool is needed, include:\n     - **Action:** The tool\'s name 
(choose one from {tool_names}).\n     - **Action Input:** The tool input in valid JSON format representing the 
keyword arguments (for example: `{{"input": "hello world", "num_beams": 5}}`).\n\n   **Example:**\n   ```\n   
Thought: I need to use a tool to help me answer the question.\n   Action: tool_name\n   Action Input: {{"input": 
"hello world", "num_beams": 5}}\n   ```\n\n2. **Waiting for the Tool Response:**  \n   - **Do Not Simulate:** Never
generate an `Observation:` line or assume the tool\'s output. After making a tool call, wait for the external tool 
to return the actual observation.\n\n3. **Final Answer (When Sufficient Information is Obtained):**  \n   When you 
have enough information to answer the question without additional tool calls, respond using one of these 
formats:\n\n   **Successful Answer:**\n   ```\n   Thought: I can answer without using any more tools.\n   Answer: 
[your answer here]\n   ```\n\n   **Unable to Answer:**\n   ```\n   Thought: I cannot answer the question with the 
provided tools.\n   Answer: Sorry, I cannot answer your query.\n   ```\n\n---\n\n### Important Reminders\n\n- 
**Always Start with a Thought:** Begin every response with a "Thought:" to describe your reasoning process.\n- 
**Valid JSON Required:** Ensure that the Action Input is formatted in valid JSON (e.g., do not use single quotes or
improper brackets).\n- **Strict No-Observation Rule:** Under no circumstances should you generate or simulate an 
`Observation:` line. Only the external tool should produce that.\n\nBy following these guidelines, you will ensure 
clear, consistent, and tool-dependent interactions.\n',
    role=<MessageRole.SYSTEM: 'system'>,
    prompt_args={
        'tool_desc': '## `power` tool.\nCalcuates the `a` in the pover of `b`\n\nParameters:\n- `a`: float\n- `b`: 
float\n## `multiply` tool.\nMultiplies `a` and `b`\n\nParameters:\n- `a`: float\n- `b`: float\n## `factorial` 
tool.\nCalcuates the factorial (!) of `a`\n\nParameters:\n- `a`: int',
        'tool_names': "['power', 'multiply', 'factorial']"
    },
    history=[
        Message(
            role=<MessageRole.USER: 'user'>,
            content='What is twice the factorial of 3?',
            tool_calls=None,
            tool_call_id=None
        )
    ],
    response=None,
    llm='gemma3:12b',
    status=<LogStatus.RUNNING: 'RUNNING'>,
    error=None
)

Trace(
    trace_id='d3975177-c1c6-4cd9-8d6b-87616dc6039a',
    parent_id='b9aadd1c-b54d-4169-925e-2b76d4ad585e',
    prompt_name='prompt',
    prompt_digest='a8edbbe0e29b507be1247289d410e3a2',
    start=datetime.datetime(2025, 6, 18, 11, 29, 11, 785574, tzinfo=datetime.timezone.utc),
    stop=datetime.datetime(2025, 6, 18, 11, 29, 13, 314929, tzinfo=datetime.timezone.utc),
    prompt='\nYou are designed to assist with a wide range of tasks—from answering questions and providing 
summaries to performing detailed analyses—by utilizing a variety of external tools. Follow these strict 
instructions to ensure correct tool usage and response formatting:\n\n---\n\n## Tools\n\n- **Tool Access:**  \n  
You have access to multiple tools: \n\n  {tool_desc}\n\n- **Execution Protocol:**  \n  - **One Step at a Time:** In
each response, you must either make a single tool call or provide a direct answer to the user.\n  - **No 
Fabrication:** You are strictly forbidden from generating any `Observation:` lines or simulating tool outputs. Do 
not assume, infer, or create tool responses—always wait for the actual output from the external tool.\n\n---\n\n## 
Output Format\n\nWhen processing a user\'s query, follow this structured format:\n\n1. **Initial Response (Tool 
Call or Direct Answer):**  \n   - **Thought:** Start every response with a clear thought outlining your reasoning. 
\n   - **Tool Call:**  \n     If you decide that a tool is needed, include:\n     - **Action:** The tool\'s name 
(choose one from {tool_names}).\n     - **Action Input:** The tool input in valid JSON format representing the 
keyword arguments (for example: `{{"input": "hello world", "num_beams": 5}}`).\n\n   **Example:**\n   ```\n   
Thought: I need to use a tool to help me answer the question.\n   Action: tool_name\n   Action Input: {{"input": 
"hello world", "num_beams": 5}}\n   ```\n\n2. **Waiting for the Tool Response:**  \n   - **Do Not Simulate:** Never
generate an `Observation:` line or assume the tool\'s output. After making a tool call, wait for the external tool 
to return the actual observation.\n\n3. **Final Answer (When Sufficient Information is Obtained):**  \n   When you 
have enough information to answer the question without additional tool calls, respond using one of these 
formats:\n\n   **Successful Answer:**\n   ```\n   Thought: I can answer without using any more tools.\n   Answer: 
[your answer here]\n   ```\n\n   **Unable to Answer:**\n   ```\n   Thought: I cannot answer the question with the 
provided tools.\n   Answer: Sorry, I cannot answer your query.\n   ```\n\n---\n\n### Important Reminders\n\n- 
**Always Start with a Thought:** Begin every response with a "Thought:" to describe your reasoning process.\n- 
**Valid JSON Required:** Ensure that the Action Input is formatted in valid JSON (e.g., do not use single quotes or
improper brackets).\n- **Strict No-Observation Rule:** Under no circumstances should you generate or simulate an 
`Observation:` line. Only the external tool should produce that.\n\nBy following these guidelines, you will ensure 
clear, consistent, and tool-dependent interactions.\n',
    role=<MessageRole.SYSTEM: 'system'>,
    prompt_args={
        'tool_desc': '## `power` tool.\nCalcuates the `a` in the pover of `b`\n\nParameters:\n- `a`: float\n- `b`: 
float\n## `multiply` tool.\nMultiplies `a` and `b`\n\nParameters:\n- `a`: float\n- `b`: float\n## `factorial` 
tool.\nCalcuates the factorial (!) of `a`\n\nParameters:\n- `a`: int',
        'tool_names': "['power', 'multiply', 'factorial']"
    },
    history=[
        Message(
            role=<MessageRole.USER: 'user'>,
            content='What is twice the factorial of 3?',
            tool_calls=None,
            tool_call_id=None
        )
    ],
    response=Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='Thought: I need to calculate the factorial of 3 first, and then multiply the result by 2. For the 
factorial calculation, I will use

Span(
    span_id='b9aadd1c-b54d-4169-925e-2b76d4ad585e',
    parent_id='c5a73a5a-d50e-4be7-a877-a9b957178c64',
    module_name='predictor',
    module_digest='5909e943adbe3a8d55d2847f2c0d8dc3',
    start=datetime.datetime(2025, 6, 18, 11, 29, 11, 785523, tzinfo=datetime.timezone.utc),
    stop=datetime.datetime(2025, 6, 18, 11, 29, 13, 315028, tzinfo=datetime.timezone.utc),
    request="What is twice the factorial of 3?\ntool_desc: ## `power` tool.\nCalcuates the `a` in the pover of 
`b`\n\nParameters:\n- `a`: float\n- `b`: float\n## `multiply` tool.\nMultiplies `a` and `b`\n\nParameters:\n- `a`: 
float\n- `b`: float\n## `factorial` tool.\nCalcuates the factorial (!) of `a`\n\nParameters:\n- `a`: 
int\ntool_names: ['power', 'multiply', 'factorial']",
    response='Thought: I need to calculate the factorial of 3 first, and then multiply the result by 2. For the 
factorial calculation, I will use the `factorial` tool.\nAction: factorial\nAction Input: {"a": 3}',
    status=<LogStatus.OK: 'OK'>,
    parameters={},
    error=None
)

...

Span(
    span_id='100b0b52-2784-4c71-8ca9-28d6a0ea210f',
    parent_id='c5a73a5a-d50e-4be7-a877-a9b957178c64',
    module_name='predictor',
    module_digest='5909e943adbe3a8d55d2847f2c0d8dc3',
    start=datetime.datetime(2025, 6, 18, 11, 29, 14, 954088, tzinfo=datetime.timezone.utc),
    stop=None,
    request="role=<MessageRole.USER: 'user'> content='Observation: 12.0' tool_calls=None 
tool_call_id=None\ntool_desc: ## `power` tool.\nCalcuates the `a` in the pover of `b`\n\nParameters:\n- `a`: 
float\n- `b`: float\n## `multiply` tool.\nMultiplies `a` and `b`\n\nParameters:\n- `a`: float\n- `b`: float\n## 
`factorial` tool.\nCalcuates the factorial (!) of `a`\n\nParameters:\n- `a`: int\ntool_names: ['power', 'multiply',
'factorial']",
    response=None,
    status=<LogStatus.RUNNING: 'RUNNING'>,
    parameters={},
    error=None
)

Trace(
    trace_id='43d2fa3b-2ebd-412f-9d28-507fba84aa87',
    parent_id='100b0b52-2784-4c71-8ca9-28d6a0ea210f',
    prompt_name='prompt',
    prompt_digest='a8edbbe0e29b507be1247289d410e3a2',
    start=datetime.datetime(2025, 6, 18, 11, 29, 14, 954284, tzinfo=datetime.timezone.utc),
    stop=None,
    prompt='\nYou are designed to assist with a wide range of tasks—from answering questions and providing 
summaries to performing detailed analyses—by utilizing a variety of external tools. Follow these strict 
instructions to ensure correct tool usage and response formatting:\n\n---\n\n## Tools\n\n- **Tool Access:**  \n  
You have access to multiple tools: \n\n  {tool_desc}\n\n- **Execution Protocol:**  \n  - **One Step at a Time:** In
each response, you must either make a single tool call or provide a direct answer to the user.\n  - **No 
Fabrication:** You are strictly forbidden from generating any `Observation:` lines or simulating tool outputs. Do 
not assume, infer, or create tool responses—always wait for the actual output from the external tool.\n\n---\n\n## 
Output Format\n\nWhen processing a user\'s query, follow this structured format:\n\n1. **Initial Response (Tool 
Call or Direct Answer):**  \n   - **Thought:** Start every response with a clear thought outlining your reasoning. 
\n   - **Tool Call:**  \n     If you decide that a tool is needed, include:\n     - **Action:** The tool\'s name 
(choose one from {tool_names}).\n     - **Action Input:** The tool input in valid JSON format representing the 
keyword arguments (for example: `{{"input": "hello world", "num_beams": 5}}`).\n\n   **Example:**\n   ```\n   
Thought: I need to use a tool to help me answer the question.\n   Action: tool_name\n   Action Input: {{"input": 
"hello world", "num_beams": 5}}\n   ```\n\n2. **Waiting for the Tool Response:**  \n   - **Do Not Simulate:** Never
generate an `Observation:` line or assume the tool\'s output. After making a tool call, wait for the external tool 
to return the actual observation.\n\n3. **Final Answer (When Sufficient Information is Obtained):**  \n   When you 
have enough information to answer the question without additional tool calls, respond using one of these 
formats:\n\n   **Successful Answer:**\n   ```\n   Thought: I can answer without using any more tools.\n   Answer: 
[your answer here]\n   ```\n\n   **Unable to Answer:**\n   ```\n   Thought: I cannot answer the question with the 
provided tools.\n   Answer: Sorry, I cannot answer your query.\n   ```\n\n---\n\n### Important Reminders\n\n- 
**Always Start with a Thought:** Begin every response with a "Thought:" to describe your reasoning process.\n- 
**Valid JSON Required:** Ensure that the Action Input is formatted in valid JSON (e.g., do not use single quotes or
improper brackets).\n- **Strict No-Observation Rule:** Under no circumstances should you generate or simulate an 
`Observation:` line. Only the external tool should produce that.\n\nBy following these guidelines, you will ensure 
clear, consistent, and tool-dependent interactions.\n',
    role=<MessageRole.SYSTEM: 'system'>,
    prompt_args={
        'tool_desc': '## `power` tool.\nCalcuates the `a` in the pover of `b`\n\nParameters:\n- `a`: float\n- `b`: 
float\n## `multiply` tool.\nMultiplies `a` and `b`\n\nParameters:\n- `a`: float\n- `b`: float\n## `factorial` 
tool.\nCalcuates the factorial (!) of `a`\n\nParameters:\n- `a`: int',
        'tool_names': "['power', 'multiply', 'factorial']"
    },
    history=[
        Message(
            role=<MessageRole.USER: 'user'>,
            content='What is twice the factorial of 3?',
            tool_calls=None,
            tool_call_id=None
        ),
        Message(
            role=<MessageRole.ASSISTANT: 'assistant'>,
            content='Thought:I need to calculate the factorial of 3 first, and then multiply the result by 2. For 
the factorial calculation, I will use the `factorial` tool.\nAction:factorial\nAction Input: {"a": 3}',
            t

Trace(
    trace_id='43d2fa3b-2ebd-412f-9d28-507fba84aa87',
    parent_id='100b0b52-2784-4c71-8ca9-28d6a0ea210f',
    prompt_name='prompt',
    prompt_digest='a8edbbe0e29b507be1247289d410e3a2',
    start=datetime.datetime(2025, 6, 18, 11, 29, 14, 954284, tzinfo=datetime.timezone.utc),
    stop=datetime.datetime(2025, 6, 18, 11, 29, 15, 861498, tzinfo=datetime.timezone.utc),
    prompt='\nYou are designed to assist with a wide range of tasks—from answering questions and providing 
summaries to performing detailed analyses—by utilizing a variety of external tools. Follow these strict 
instructions to ensure correct tool usage and response formatting:\n\n---\n\n## Tools\n\n- **Tool Access:**  \n  
You have access to multiple tools: \n\n  {tool_desc}\n\n- **Execution Protocol:**  \n  - **One Step at a Time:** In
each response, you must either make a single tool call or provide a direct answer to the user.\n  - **No 
Fabrication:** You are strictly forbidden from generating any `Observation:` lines or simulating tool outputs. Do 
not assume, infer, or create tool responses—always wait for the actual output from the external tool.\n\n---\n\n## 
Output Format\n\nWhen processing a user\'s query, follow this structured format:\n\n1. **Initial Response (Tool 
Call or Direct Answer):**  \n   - **Thought:** Start every response with a clear thought outlining your reasoning. 
\n   - **Tool Call:**  \n     If you decide that a tool is needed, include:\n     - **Action:** The tool\'s name 
(choose one from {tool_names}).\n     - **Action Input:** The tool input in valid JSON format representing the 
keyword arguments (for example: `{{"input": "hello world", "num_beams": 5}}`).\n\n   **Example:**\n   ```\n   
Thought: I need to use a tool to help me answer the question.\n   Action: tool_name\n   Action Input: {{"input": 
"hello world", "num_beams": 5}}\n   ```\n\n2. **Waiting for the Tool Response:**  \n   - **Do Not Simulate:** Never
generate an `Observation:` line or assume the tool\'s output. After making a tool call, wait for the external tool 
to return the actual observation.\n\n3. **Final Answer (When Sufficient Information is Obtained):**  \n   When you 
have enough information to answer the question without additional tool calls, respond using one of these 
formats:\n\n   **Successful Answer:**\n   ```\n   Thought: I can answer without using any more tools.\n   Answer: 
[your answer here]\n   ```\n\n   **Unable to Answer:**\n   ```\n   Thought: I cannot answer the question with the 
provided tools.\n   Answer: Sorry, I cannot answer your query.\n   ```\n\n---\n\n### Important Reminders\n\n- 
**Always Start with a Thought:** Begin every response with a "Thought:" to describe your reasoning process.\n- 
**Valid JSON Required:** Ensure that the Action Input is formatted in valid JSON (e.g., do not use single quotes or
improper brackets).\n- **Strict No-Observation Rule:** Under no circumstances should you generate or simulate an 
`Observation:` line. Only the external tool should produce that.\n\nBy following these guidelines, you will ensure 
clear, consistent, and tool-dependent interactions.\n',
    role=<MessageRole.SYSTEM: 'system'>,
    prompt_args={
        'tool_desc': '## `power` tool.\nCalcuates the `a` in the pover of `b`\n\nParameters:\n- `a`: float\n- `b`: 
float\n## `multiply` tool.\nMultiplies `a` and `b`\n\nParameters:\n- `a`: float\n- `b`: float\n## `factorial` 
tool.\nCalcuates the factorial (!) of `a`\n\nParameters:\n- `a`: int',
        'tool_names': "['power', 'multiply', 'factorial']"
    },
    history=[
        Message(
            role=<MessageRole.USER: 'user'>,
            content='What is twice the factorial of 3?',
            tool_calls=None,
            tool_call_id=None
        ),
        Message(
            role=<MessageRole.ASSISTANT: 'assistant'>,
            content='Thought:I need to calculate the factorial of 3 first, and then multiply the result by 2. For 
the factorial calculation, I will use the

Span(
    span_id='100b0b52-2784-4c71-8ca9-28d6a0ea210f',
    parent_id='c5a73a5a-d50e-4be7-a877-a9b957178c64',
    module_name='predictor',
    module_digest='5909e943adbe3a8d55d2847f2c0d8dc3',
    start=datetime.datetime(2025, 6, 18, 11, 29, 14, 954088, tzinfo=datetime.timezone.utc),
    stop=datetime.datetime(2025, 6, 18, 11, 29, 15, 861626, tzinfo=datetime.timezone.utc),
    request="role=<MessageRole.USER: 'user'> content='Observation: 12.0' tool_calls=None 
tool_call_id=None\ntool_desc: ## `power` tool.\nCalcuates the `a` in the pover of `b`\n\nParameters:\n- `a`: 
float\n- `b`: float\n## `multiply` tool.\nMultiplies `a` and `b`\n\nParameters:\n- `a`: float\n- `b`: float\n## 
`factorial` tool.\nCalcuates the factorial (!) of `a`\n\nParameters:\n- `a`: int\ntool_names: ['power', 'multiply',
'factorial']",
    response='Thought: I have the result of twice the factorial of 3, which is 12.\nAnswer: 12.0',
    status=<LogStatus.OK: 'OK'>,
    parameters={},
    error=None
)

Span(
    span_id='c5a73a5a-d50e-4be7-a877-a9b957178c64',
    parent_id=None,
    module_name='root',
    module_digest='a31de4f795d2eed6e3dae3ba1896d4bc',
    start=datetime.datetime(2025, 6, 18, 11, 29, 11, 785450, tzinfo=datetime.timezone.utc),
    stop=datetime.datetime(2025, 6, 18, 11, 29, 15, 861765, tzinfo=datetime.timezone.utc),
    request='What is twice the factorial of 3?',
    response='12.0',
    status=<LogStatus.OK: 'OK'>,
    parameters={},
    error=None
)